In [4]:
import sqlalchemy

In [9]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///test", echo = True)

print(engine)

Engine(sqlite:///test)


In [10]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

In [11]:
metadata = sqlalchemy.MetaData()

In [12]:
artist = Table("Artist",metadata,
               Column("id",Integer, primary_key = True),
               Column("name",String, nullable=False),
               extend_existing=True)

In [14]:
album = Table("Album",metadata,
               Column("id",Integer, primary_key = True),
               Column("title",String, nullable=False),
               Column("artist_id",Integer, ForeignKey("Artist.id")),
               extend_existing=True)

In [15]:
genre = Table("Genre",metadata,
               Column("id",Integer, primary_key = True),
               Column("name",String, nullable=False),
               extend_existing=True)

In [16]:
track = Table("Track",metadata,
               Column("id",Integer, primary_key = True),
               Column("title",String, nullable=False),
               Column("length",Integer, nullable=False),
               Column("rating",Integer, nullable=False),
               Column("count",Integer, nullable=False),
               Column("album_id",Integer, ForeignKey("Album.id")),
               Column("genre_id",Integer, ForeignKey("Genre.id")),
               extend_existing=True)

In [17]:
metadata.create_all(engine)

2019-07-09 13:39:17,756 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-09 13:39:17,757 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:39:17,759 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-09 13:39:17,759 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:39:17,761 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Artist")
2019-07-09 13:39:17,762 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:39:17,764 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2019-07-09 13:39:17,765 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:39:17,766 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Genre")
2019-07-09 13:39:17,767 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:39:17,768 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Track")
2019-07-09 13:39:17,769 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:39:17,770 INFO sqlalchemy.engin

In [19]:
tables = metadata.tables    # 메모리에 있는 값
for table in tables:
    print(table)

Artist
Album
Genre
Track


In [22]:
conn = engine.connect()
conn

In [23]:
for table in engine.table_names():   # 엔진에 있는 값
    print(table)

2019-07-09 13:45:29,585 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-07-09 13:45:29,586 INFO sqlalchemy.engine.base.Engine ()
Album
Artist
Genre
Track


Insert

In [ ]:
conn.execute(artist.insert(),[
    {"name":"Led Zepplin"},
    {"name":"AC/DC"}
])

In [25]:
conn.execute(album.insert(),[
    {"title":"IV", "artist_id":1},
    {"title":"who made who", "artist_id":2},
])

2019-07-09 13:47:54,236 INFO sqlalchemy.engine.base.Engine INSERT INTO "Album" (title, artist_id) VALUES (?, ?)
2019-07-09 13:47:54,237 INFO sqlalchemy.engine.base.Engine (('IV', 1), ('who made who', 2))
2019-07-09 13:47:54,240 INFO sqlalchemy.engine.base.Engine COMMIT


In [26]:
conn.execute(genre.insert(),[
    {"name":"Rock"},
    {"name":"Metal"}   
])

2019-07-09 13:48:34,524 INFO sqlalchemy.engine.base.Engine INSERT INTO "Genre" (name) VALUES (?)
2019-07-09 13:48:34,525 INFO sqlalchemy.engine.base.Engine (('Rock',), ('Metal',))
2019-07-09 13:48:34,529 INFO sqlalchemy.engine.base.Engine COMMIT


In [28]:
conn.execute(track.insert(),[
    {"title":"Black Dog","rating":5, "length":297, "count":0, "album_id":1, "genre_id":1},
    {"title":"Stair Way","rating":5, "length":456, "count":0, "album_id":1, "genre_id":1},
    {"title":"about to rock","rating":5, "length":123, "count":0, "album_id":2, "genre_id":2},
    {"title":"who made who","rating":5, "length":897, "count":0, "album_id":2, "genre_id":2}


])

2019-07-09 13:51:18,197 INFO sqlalchemy.engine.base.Engine INSERT INTO "Track" (title, length, rating, count, album_id, genre_id) VALUES (?, ?, ?, ?, ?, ?)
2019-07-09 13:51:18,198 INFO sqlalchemy.engine.base.Engine (('Black Dog', 297, 5, 0, 1, 1), ('Stair Way', 456, 5, 0, 1, 1), ('about to rock', 123, 5, 0, 2, 2), ('who made who', 897, 5, 0, 2, 2))
2019-07-09 13:51:18,201 INFO sqlalchemy.engine.base.Engine COMMIT


### Select

In [29]:
artistResult = conn.execute(artist.select())
for row in artistResult:
    print(row)


2019-07-09 14:08:56,491 INFO sqlalchemy.engine.base.Engine SELECT "Artist".id, "Artist".name 
FROM "Artist"
2019-07-09 14:08:56,492 INFO sqlalchemy.engine.base.Engine ()
(1, 'Led Zepplin')
(2, 'AC/DC')


### Update

In [ ]:
from sqlalchemy import update

conn.execute(trackupdate().values(genre_id=2).where(track.c.id==2))

In [ ]:
# 알아서 해보기

### Where

### Join

In [33]:
print(track.join(album))
print(track.join(album).join(genre))
print(track.join(album).join(artist))
print(track.join(album).join(genre).join(artist))

result = conn.execute(select([track.c.title, album.c.title, genre.c.name, artist.c.name]) \
                      .select_from(track.join(album).join(genre).join(artist)))

for row in result.fetchall():
    print(row)


"Track" JOIN "Album" ON "Album".id = "Track".album_id
"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id
"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Artist" ON "Artist".id = "Album".artist_id
"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id


NameError: name 'select' is not defined

In [ ]:
result = conn.execute(track.select().select_from(track.join(album).join(genre).join(artist)))
                        .select(
                        .s)

In [35]:
# 엔진은 살아있고, 메타데이터는 사라짐. 

conn.close()
metadata.clear()

In [37]:
metadata.tables  # 메타데이터 사라짐

immutabledict({})

In [39]:
tables = metadata.tables     

for table in engine.table_names():     # 엔진에는 살아있음.
    print(table)

2019-07-09 14:21:41,322 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-07-09 14:21:41,323 INFO sqlalchemy.engine.base.Engine ()
Album
Artist
Genre
Track


다시 불러오기

In [41]:
metadata = MetaData(bind = engine, reflect = True)
metadata.reflect(bind=engine)

2019-07-09 14:24:02,682 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-07-09 14:24:02,683 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:24:02,686 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2019-07-09 14:24:02,687 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:24:02,689 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Album' AND type = 'table'
2019-07-09 14:24:02,690 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:24:02,692 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("Album")
2019-07-09 14:24:02,693 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:24:02,694 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Album' AND type = 'table'
2019-07-09 14:24:02,695 INFO sqlalchemy.engine.base.Engine ()
2019-07

C:\Users\uj274\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.
  """Entry point for launching an IPython kernel.


In [42]:
metadata.tables

immutabledict({'Album': Table('Album', MetaData(bind=Engine(sqlite:///test)), Column('id', INTEGER(), table=<Album>, primary_key=True, nullable=False), Column('title', VARCHAR(), table=<Album>, nullable=False), Column('artist_id', INTEGER(), ForeignKey('Artist.id'), table=<Album>), schema=None), 'Artist': Table('Artist', MetaData(bind=Engine(sqlite:///test)), Column('id', INTEGER(), table=<Artist>, primary_key=True, nullable=False), Column('name', VARCHAR(), table=<Artist>, nullable=False), schema=None), 'Genre': Table('Genre', MetaData(bind=Engine(sqlite:///test)), Column('id', INTEGER(), table=<Genre>, primary_key=True, nullable=False), Column('name', VARCHAR(), table=<Genre>, nullable=False), schema=None), 'Track': Table('Track', MetaData(bind=Engine(sqlite:///test)), Column('id', INTEGER(), table=<Track>, primary_key=True, nullable=False), Column('title', VARCHAR(), table=<Track>, nullable=False), Column('length', INTEGER(), table=<Track>, nullable=False), Column('rating', INTEGER(

In [45]:
class User(base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column('passwd', String)
    
    def __repr__(self):
        return f'<Table User(name={self.name}, fullname={self.fullname}, password={self.password}>'

NameError: name 'base' is not defined